<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/%EB%85%B8%EC%9D%B4)%EC%A6%9D%EB%B6%84G_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%91%EB%B6%84%EB%A5%98_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 177 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 12.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=44805e4d0d514a2e36eaf45d6b5c2822d75310f80792e167cb1328934a4fa65c
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 14.0 MB/s 
     |████████████████████████████████| 769 kB 15.4 MB/s 
     |████████████████████████████████| 895 kB 88.3 MB/s 
     |████████████████████████████████| 3.0 MB 85.7 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-1ltszq_9
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본   산업분류자동화	'Colab Notebooks'  'My Drive'


In [10]:
# spell check한 data
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/G_spell_check.csv', sep = ",", encoding = "UTF-8")

In [11]:
df[['digit_2','digit_3']].value_counts().sort_index()

digit_2  digit_3
45       451         2268
         452         4696
         453          471
46       461         2328
         462         2585
         463        17873
         464        18453
         465        14652
         466         8154
         467        12096
         468          694
47       471        26189
         472        25668
         473         9312
         474        36453
         475        13031
         476         7188
         477         4577
         478        30566
         479         9218
dtype: int64

## 데이터 전처리

In [12]:
df['digit_2'].value_counts()

47    162202
46     76835
45      7435
Name: digit_2, dtype: int64

In [14]:
df.loc[(df['digit_2'] == 45), 'digit_2'] = 0  
df.loc[(df['digit_2'] == 46), 'digit_2'] = 1  
df.loc[(df['digit_2'] == 47), 'digit_2'] = 2  

In [15]:
data_list = []
for q, label in zip(df['text'], df['digit_2'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['매장에서 일반인 대상으로 의류소매', '2'], ['매장에서 일반인대상으로 의류소매', '2'], ['사업장에서 일반인대상 의류판매', '2'], ['일반고객에게 신발매장에서 일반인을 대상으로 신발', '2'], ['일반소비자 가정용 직물제품소매점 커튼 방석 침구류등 소매', '2']]


## Train data & test data

In [19]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [21]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [22]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [23]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [24]:
data_train[0]

(array([   2, 1998, 6903, 2822, 6168, 6877, 6897,  517, 5859, 6168,  517,
        5330, 5452, 6732,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [25]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [27]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [28]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/3466 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.1047662496566772 train acc 0.375
epoch 1 batch id 201 loss 0.3733539581298828 train acc 0.6725746268656716
epoch 1 batch id 401 loss 0.20795685052871704 train acc 0.7987453241895262
epoch 1 batch id 601 loss 0.18411730229854584 train acc 0.8496516222961731
epoch 1 batch id 801 loss 0.1796756535768509 train acc 0.8759363295880149
epoch 1 batch id 1001 loss 0.05211039260029793 train acc 0.8922795954045954
epoch 1 batch id 1201 loss 0.1849268078804016 train acc 0.9029194421315571
epoch 1 batch id 1401 loss 0.06673295795917511 train acc 0.9110122234118487
epoch 1 batch id 1601 loss 0.07426914572715759 train acc 0.9167707682698314
epoch 1 batch id 1801 loss 0.07808792591094971 train acc 0.9217361882287618
epoch 1 batch id 2001 loss 0.0457560271024704 train acc 0.9255216141929036
epoch 1 batch id 2201 loss 0.06458444148302078 train acc 0.9288462630622444
epoch 1 batch id 2401 loss 0.13089455664157867 train acc 0.9317146501457726
epoch 1 batch id 2601 loss 0.08842293

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9666450777202072


  0%|          | 0/3466 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.02104589343070984 train acc 1.0
epoch 5 batch id 201 loss 0.180540531873703 train acc 0.9783115671641791
epoch 5 batch id 401 loss 0.07927877455949783 train acc 0.9793874688279302
epoch 5 batch id 601 loss 0.06522153317928314 train acc 0.9789933444259568
epoch 5 batch id 801 loss 0.14718171954154968 train acc 0.9789715980024969
epoch 5 batch id 1001 loss 0.010120443999767303 train acc 0.9794736513486514
epoch 5 batch id 1201 loss 0.13222528994083405 train acc 0.9794572231473772
epoch 5 batch id 1401 loss 0.024278098717331886 train acc 0.9796239293361885
epoch 5 batch id 1601 loss 0.014298709109425545 train acc 0.979690427857589
epoch 5 batch id 1801 loss 0.01077316515147686 train acc 0.9799330233203776
epoch 5 batch id 2001 loss 0.010984186083078384 train acc 0.9799631434282858
epoch 5 batch id 2201 loss 0.011563747189939022 train acc 0.9803072467060427
epoch 5 batch id 2401 loss 0.13132692873477936 train acc 0.9804768846314036
epoch 5 batch id 2601 loss 0.075

  0%|          | 0/386 [00:00<?, ?it/s]

epoch 5 test acc 0.9665641191709845


In [29]:
#학습 모델 저장
torch.save(model, 'G_inc_jung_model.pt')